<a href="https://colab.research.google.com/github/llawlaw23/Recommendation-System/blob/main/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import gc
import re

In [ ]:
props1 = pd.read_csv("item_properties_part1.1.csv")
props2 = pd.read_csv("item_properties_part2.csv")
props = pd.concat([props1, props2])

Mapping itemid to categoryid from Properties Dataset

In our dataset, the category of each item is stored within the properties table, specifically in rows where the property column has the value "categoryid". To extract a clean mapping of itemid → categoryid, we perform the following steps:

Filter the dataset to keep only rows where property == "categoryid".

Rename the value_n column to categoryid for clarity.

Convert the category IDs to integers to facilitate merging with other tables.


In [ ]:
df_items = props[props["property"] == "categoryid"][["itemid", "value"]]

In [ ]:
df_items = df_items.rename(columns={"value": "categoryid"})

In [ ]:
df_items["categoryid"] = df_items["categoryid"].astype("Int64")

In [ ]:
# saved the clean csv as an new file in the cleaned folder and delete the previous one
df_items.to_csv("df_item.csv", index = False)
del df_items; gc.collect()

In [ ]:
props.shape

(20275902, 4)

In [ ]:
props.head()

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513


In [ ]:
events = pd.read_csv("events.csv")
cats = pd.read_csv("category_tree.csv")

### Data Understanding & Preprocessing

#### 1. Data Overview
- Brief description of the dataset ( size, number of features, number of columns and rows, and observations)
- Types of variables (numerical, categorical, datetime, etc.)
- Initial observations about the data (e.g., imbalance, missing data, outliers)

#### 2. Data Quality Checks
- Check for missing values and filling or dropping them.
- dropping columns.
- checking for outliers.

In [ ]:
events["timestamp"] = pd.to_datetime(events["timestamp"], unit = "ms")

In [ ]:
events.shape

(2756101, 5)

In [ ]:
events.isnull().sum()

,0
timestamp,0
visitorid,0
event,0
itemid,0
transactionid,2733644


In [ ]:
events = events.drop(columns = ["transactionid"])

In [ ]:
events.head()

,timestamp,visitorid,event,itemid
0,2015-06-02 05:02:12.117,257597,view,355908
1,2015-06-02 05:50:14.164,992329,view,248676
2,2015-06-02 05:13:19.827,111016,view,318965
3,2015-06-02 05:12:35.914,483717,view,253185
4,2015-06-02 05:02:17.106,951259,view,367447


In [ ]:
# map view = 1 add to chart = 3 and purchase  = 5, replacing it in the old column. dataype to int
event_weights = {"view": 1, "addtocart": 2, "transaction": 3}
events["event"] = events["event"].map(event_weights)

In [ ]:
# change datatype of event to int
events["event"] = events["event"].fillna(0).astype(int)

In [ ]:
events.head(20)

,timestamp,visitorid,event,itemid
0,2015-06-02 05:02:12.117,257597,1,355908
1,2015-06-02 05:50:14.164,992329,1,248676
2,2015-06-02 05:13:19.827,111016,1,318965
3,2015-06-02 05:12:35.914,483717,1,253185
4,2015-06-02 05:02:17.106,951259,1,367447
5,2015-06-02 05:48:06.234,972639,1,22556
6,2015-06-02 05:12:03.240,810725,1,443030
7,2015-06-02 05:34:51.897,794181,1,439202
8,2015-06-02 04:54:59.221,824915,1,428805
9,2015-06-02 05:00:04.592,339335,1,82389


In [ ]:
# min in visitorid
events["visitorid"].min()

0

checking for outliers and clearing them

In [ ]:
visitor_event = events.groupby("visitorid")["event"].count()

In [ ]:
# print in descending order
visitor_event = visitor_event.sort_values(ascending = False)

In [ ]:
visitor_event.head(20)

,event
visitorid,
1150086,7757
530559,4328
152963,3024
895999,2474
163561,2410
371606,2345
286616,2252
684514,2246
892013,2024


In [ ]:
Q1 = visitor_event.quantile(0.25)
Q3 = visitor_event.quantile(0.75)
IQR = Q3 - Q1

In [ ]:
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

In [ ]:
upper_bound

np.float64(3.5)

In [ ]:
lower_bound

np.float64(-0.5)

In [ ]:
outlier_visitor_ids = visitor_event[(visitor_event < lower_bound) | (visitor_event > upper_bound)].index

In [ ]:
is_outlier = events["visitorid"].isin(outlier_visitor_ids)

In [ ]:
print (is_outlier)

0          False
1           True
2          False
3          False
4          False
           ...  
2756096    False
2756097    False
2756098    False
2756099    False
2756100    False
Name: visitorid, Length: 2756101, dtype: bool


In [ ]:
events_F = events[~is_outlier]

In [ ]:
events_F.head()

,timestamp,visitorid,event,itemid
0,2015-06-02 05:02:12.117,257597,1,355908
2,2015-06-02 05:13:19.827,111016,1,318965
3,2015-06-02 05:12:35.914,483717,1,253185
4,2015-06-02 05:02:17.106,951259,1,367447
5,2015-06-02 05:48:06.234,972639,1,22556


In [ ]:
events_F.shape

(1652380, 4)

In [ ]:
props.head()

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513


In [ ]:
itemid_value = props.groupby("itemid")["value"].count()

In [ ]:
Q1 = itemid_value.quantile(0.25)
Q3 = itemid_value.quantile(0.75)
IQR = Q3 - Q1

In [ ]:
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

In [ ]:
outlier_itemid_value = itemid_value[(itemid_value < lower_bound) | (itemid_value > upper_bound)].index

In [ ]:
is_outlier = props["itemid"].isin(outlier_itemid_value)

In [ ]:
props = props[~is_outlier]

In [ ]:
props.shape

(17520979, 4)

In [ ]:
# clean the value column and keep values with n attached to them and add them if they are multiple
def sum_n_values(text):
    nums = re.findall(r"n([\d\.]+)", str(text))
    nums = [float(x) for x in nums]
    return sum(nums) if nums else None

props["value_n"] = props["value"].apply(sum_n_values)

In [ ]:
props = props.drop(columns = ["value"])

In [ ]:
props.head()

,timestamp,itemid,property,value_n
0,1435460400000,460429,categoryid,NaN
1,1441508400000,206783,888,277.2
2,1439089200000,395014,400,1272.0
3,1431226800000,59481,790,15360.0
4,1431831600000,156781,917,NaN


In [ ]:
# drop null values in value_n column
props = props.dropna(subset = ["value_n"])

In [ ]:
props["timestamp"] = pd.to_datetime(props["timestamp"], unit = "ms")

In [ ]:
props.shape

(4300636, 4)

In [ ]:
cats.isnull().sum()

,0
categoryid,0
parentid,25


In [ ]:
cats = cats.dropna()

In [ ]:
# saved the clean csv as an new file in the cleaned folder and delete the previous one
props.to_csv("props_cleaned.csv", index = False)
del props; gc.collect()

In [ ]:
# saved the clean csv as an new file in the cleaned folder and delete the previous one
events_F.to_csv("events_cleaned.csv", index = False)
del events_F; gc.collect()

In [ ]:
# saved the clean csv as an new file in the cleaned folder and delete the previous one
cats.to_csv("cats_cleaned.csv", index = False)
del cats; gc.collect()

In [ ]:
import pandas as pd

In [ ]:
# Loading cleaned csv files
props_cleaned = pd.read_csv("props_cleaned.csv")

events_cleaned = pd.read_csv("events_cleaned.csv")

cats_cleaned = pd.read_csv("cats_cleaned.csv")

df_items = pd.read_csv("df_item.csv")

In [ ]:
events_cleaned.head()

,timestamp,visitorid,event,itemid,year_month
0,2015-06-02 05:02:12.117,257597,1,355908,2015-06
1,2015-06-02 05:13:19.827,111016,1,318965,2015-06
2,2015-06-02 05:12:35.914,483717,1,253185,2015-06
3,2015-06-02 05:02:17.106,951259,1,367447,2015-06
4,2015-06-02 05:48:06.234,972639,1,22556,2015-06


In [ ]:
df_items.head()

,itemid,categoryid,parentid
0,460429,1338,1278.0
1,281245,1277,312.0
2,35575,1059,1696.0
3,8313,1147,1027.0
4,55102,47,381.0


In [ ]:
df_items = df_items.drop_duplicates(subset = "itemid").reset_index(drop = True)

In [ ]:
# merging df_items and cats_cleaned
df_items = df_items.merge(cats_cleaned, on = "categoryid", how = "left")
# print(df_items.head())

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

RNG = np.random.default_rng(42)

In [ ]:
# Pivot numeric item properties into wide matrix: rows = itemid, cols=property
# We aggregate with mean to handle multiple rows per (item, property)
item_prop_wide = (
    props_cleaned
    .pivot_table(index =  "itemid", columns = "property", values = "value_n", aggfunc = "mean")
    .replace([np.inf, -np.inf], np.nan)
)

In [ ]:
# One-hot encode category and parent
cat_ohe = pd.get_dummies(df_items[["categoryid","parentid"]].astype("string"),
                         columns=["categoryid","parentid"], prefix = ["cat","par"])

In [ ]:
# Join: itemid + OHE
item_cat_feat = pd.concat([df_items[["itemid"]], cat_ohe], axis = 1).set_index("itemid")

In [ ]:
item_features = (
    item_prop_wide
    .merge(item_cat_feat, left_index = True, right_index = True, how = "outer")
    .fillna(0)
)

In [ ]:
# Keep itemid as a column for convenience in later merges
item_features = item_features.reset_index()
# item_features.head()

For every user, it computes the set difference of all items vs. the user’s interacted items.

In [ ]:
# Positive labels
pos = (
    events_cleaned[["visitorid","itemid"]]
    .drop_duplicates()
    .assign(label=1)
)

# Universe of items to sample negatives from
all_items = item_features["itemid"].unique()

In [ ]:
# Negative sampling: for each user, sample as many negatives as positives
neg_rows = []
for u, grp in pos.groupby("visitorid"):
    interacted = grp["itemid"].unique()
    candidates = np.setdiff1d(all_items, interacted)
    k = len(interacted) if len(candidates) >= len(interacted) else len(candidates)
    sampled = RNG.choice(candidates, size=k, replace=False)
    neg_rows.extend([(u, i, 0) for i in sampled])

neg = pd.DataFrame(neg_rows, columns=["visitorid","itemid","label"])

KeyboardInterrupt: 

In [ ]:
# Combine
labeled_ui = pd.concat([pos, neg], ignore_index=True)
labeled_ui.head(), labeled_ui["label"].value_counts()

In [ ]:
# Basic user stats from interactions (can be extended)
user_stats = (
    events_cleaned.groupby("visitorid")
    .agg(
        ui_events=("itemid","count"),
        ui_unique_items=("itemid","nunique"),
    )
    .reset_index()
)

In [ ]:
# Merge user stats into the training set
train_df = (
    labeled_ui
    .merge(user_stats, on = "visitorid", how = "left")
    .merge(item_features, on = "itemid", how = "left")
    .fillna(0)
)

train_df.head()

model Train / Test split + model training

In [ ]:
# Features/labels
train_df["user_idx"] = train_df["visitorid"].astype("category").cat.codes
train_df["item_idx"] = train_df["itemid"].astype("category").cat.codes

In [ ]:
# Automatically select numeric columns
numeric_cols = train_df.select_dtypes(include=np.number).columns.tolist()

# Remove the label column from features
numeric_cols.remove("label")

# Feature matrix
X = train_df[numeric_cols]

# Target label
y = train_df["label"]

In [ ]:
print("Numeric feature columns:", numeric_cols)
print("Feature matrix shape:", X.shape)
print("Label vector shape:", y.shape)

Numeric feature columns: ['visitorid', 'itemid', 'user_idx', 'item_idx']
Feature matrix shape: (1496877, 4)
Label vector shape: (1496877,)


In [ ]:
# Features/labels
drop_cols = ["visitorid","itemid","label"]
X = train_df.drop(columns = drop_cols)
y = train_df["label"].astype(int)

In [ ]:
# Scale (tree models don't require scaling, but it won't hurt)
scaler = StandardScaler(with_mean = False)
X_scaled = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size = 0.2, random_state = 42, stratify = y
)

In [ ]:
# RandomForest
clf = RandomForestClassifier(
    n_estimators = 400,
    max_depth = None,
    min_samples_leaf = 2,
    n_jobs = -1,
    random_state = 42
)
clf.fit(X_train, y_train)

RandomForestClassifier(min_samples_leaf=2, n_estimators=400, n_jobs=-1,
                       random_state=42)

In [ ]:
# Quick AUC sanity check
probs = clf.predict_proba(X_test)[:,1]
auc = roc_auc_score(y_test, probs)
print(f"AUC: {auc:.4f}")

IndexError: index 1 is out of bounds for axis 1 with size 1

In [ ]:
recs_df = pd.DataFrame(
    [(user, item, score) for user, recs in user_recs.items() for item, score in recs],
    columns = ["visitor_idx", "item_idx", "score"]
)

In [ ]:
def recommend_for_user(user_id, top_n=10):
    """
    Score all candidate items the user hasn't interacted with
    and return top-N by predicted probability.
    """
    # Items user already has
    already = set(pos.loc[pos["visitorid"] == user_id, "itemid"].values)

    # Candidate items = all items not already seen
    candidates = item_features[~item_features["itemid"].isin(already)]["itemid"].values
    if len(candidates) == 0:
        return pd.DataFrame(columns=["itemid","score"])

    # Build feature rows: user features + item features
    user_row = user_stats[user_stats["visitorid"] == user_id]
    if user_row.empty:
        # If new user: zeros
        user_block = pd.DataFrame({"ui_events":[0], "ui_unique_items":[0]})
    else:
        user_block = user_row[["ui_events","ui_unique_items"]].reset_index(drop = True)

    # Broadcast user features to candidate items
    user_block = pd.concat([user_block]*len(candidates), ignore_index = True)
    items_block = item_features[item_features["itemid"].isin(candidates)].reset_index(drop = True)

    # Align columns to training feature order
    feats = pd.concat([user_block, items_block.drop(columns = ["itemid"])], axis = 1)
    feats = feats.reindex(columns = X.columns, fill_value = 0)

    # Scale + predict
    feats_scaled = scaler.transform(feats)
    scores = clf.predict_proba(feats_scaled)[:,1]

    out = pd.DataFrame({"itemid": candidates, "score": scores})
    return out.sort_values("score", ascending=False).head(top_n)

# Example usage:
print(recommend_for_user(257597, top_n=5))


### Data Visualization and Analysing Business Question

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

Q1
- How many unique visitors in a month?

In [ ]:
events_cleaned["timestamp"] = pd.to_datetime(events_cleaned["timestamp"])

In [ ]:
events_cleaned.head()

,timestamp,visitorid,event,itemid
0,2015-06-02 05:02:12.117,257597,1,355908
1,2015-06-02 05:13:19.827,111016,1,318965
2,2015-06-02 05:12:35.914,483717,1,253185
3,2015-06-02 05:02:17.106,951259,1,367447
4,2015-06-02 05:48:06.234,972639,1,22556


In [ ]:
events_cleaned["year_month"] = events_cleaned["timestamp"].dt.to_period("M")

In [ ]:
monthly_visitors = events_cleaned.groupby("year_month")["visitorid"].nunique().reset_index()

In [ ]:

print(monthly_visitors.head(10))

  year_month  visitorid
0    2015-05     274302
1    2015-06     276649
2    2015-07     337179
3    2015-08     279387
4    2015-09     155952


In [ ]:
monthly_visitors["year_month"] = monthly_visitors["year_month"].astype(str)

In [ ]:
color_map = {
    "2015-05": "#F564A9",
    "Jun 2015": "#98A1BC",
    "Jul 2015": "#DED3C4",
    "Aug 2015": "#264653",
    "Sep 2015": "#670D2F"
}

fig = px.bar(
    monthly_visitors,
    x = "year_month",
    y = "visitorid",
    color_discrete_map = color_map,
    title = "Unique Visitors per Month",
    labels = {"year_month": "Month", "visitorid": "Unique Visitors"},
    text = "visitorid"
)
fig.update_layout(
    legend_title_text = "Month",
    xaxis_title = "Month",
    yaxis = dict(
        showticklabels = False
    )
)

fig.show()

Q2
- What patterns exist in user-item interactions over time?

In [ ]:
monthly_interactions = (
    events_cleaned.groupby("year_month")["visitorid"]
    .count()
    .reset_index(name="total_interactions")
)

In [ ]:
monthly_interactions["year_month"] = monthly_interactions["year_month"].dt.to_timestamp()

In [ ]:

fig = px.bar(
    monthly_interactions,
    x = "year_month",
    y = "total_interactions",
    title = "Monthly User-Item Interactions",
    labels = {"year_month": "Month", "total_interactions": "Number of Interactions"},
    text = "total_interactions"
)

fig.update_layout(
    legend_title_text = "Month",
    xaxis_title = "Month",
    yaxis = dict(
        showticklabels = False
    )
)

fig.show()


Q3
- Which items have the highest property values overall?

In [ ]:
item_values = props_cleaned.groupby("itemid")["value_n"].max().reset_index()

In [ ]:
fig = px.bar(item_values.sort_values("value_n", ascending=False).head(10),
             x = "Itemid", y = "value",
             title = "Top Items by Highest Property Value",
             labels = {"value_n":"Max Value"})

fig.update_layout(
    legend_title_text = "Itemid",
    xaxis_title = "Itemid",
    yaxis = dict(
        showticklabels = False
    )
)
fig.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['itemid', 'value_n'] but received: Itemid

Q4
- Which properties occur most frequently across items?

In [ ]:
prop_counts = props_cleaned["property"].value_counts().reset_index()
prop_counts.columns = ["property", "count"]

In [ ]:
fig = px.bar(prop_counts, x = "Property", y = "Count",
             title = "Most Common Properties",
             labels = {"count":"Frequency"})
fig.update_layout(
    legend_title_text = "Property",
    xaxis_title = "Property",
    yaxis = dict(
        showticklabels = False
    )
)
fig.show()

Q5
- Which visitors interacted with the highest number of unique items?

In [ ]:
user_items = events_cleaned.groupby("visitorid")["itemid"].nunique().sort_values(ascending = False)

In [ ]:
fig = px.bar(user_items, x = "Visitorid", y = "Itemid",
             title = "Top Visitors by Unique Items",
             labels = {"itemid":"Unique Items", "visitorid":"Visitor ID"})
fig.update_layout(
    legend_title_text = "Visitorid",
    xaxis_title = "Visitorid",
    yaxis = dict(
        showticklabels = False
    )
)

fig.show()

Q6
- What are the most frequently interacted items across all visitors?

In [ ]:
item_counts = events_cleaned["itemid"].value_counts().reset_index()
item_counts.columns = ["itemid", "count"]

In [ ]:
fig = px.bar(item_counts.head(10), x = "Itemid", y = "Count",
             title = "Most Popular Items",
             labels = {"count":"Number of Interactions", "itemid":"Item ID"})
fig.update_layout(
    legend_title_text = "Itemid",
    xaxis_title = "Itemid",
    yaxis = dict(
        showticklabels = False
    )
)
fig.show()

Q7
- How do the types of events vary across items and visitors?

In [ ]:
event_counts = events_cleaned["event"].value_counts().reset_index()
event_counts.columns = ["event", "count"]

In [ ]:
fig = px.pie(event_counts, names = "Event", values = "Count",
             title="Distribution of Event Types")
fig.update_layout(
    legend_title_text = "Event",
    xaxis_title = "Event",
    yaxis = dict(
        showticklabels = False
    )
)
fig.show()